# 6.3 Geographic Visualization 

### This script contains the following:
#### 1. Import data and libraries

### 1. Import data and libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
# This command propts matplotlib visuals to appear in the notebook 

%matplotlib inline

In [3]:
# set path

path = r'/Users/kurtson/Desktop/House Sales'

In [4]:
df = pd.read_csv(os.path.join(path, 'Prepared Data', 'cleaned_home.csv'), index_col = False)

In [5]:
df.head()

,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,year_built,year_renovated,zipcode,latitude,longitude
0,0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045


In [6]:
# Import ".json" file for the U.S. 

country_geo = r'/Users/kurtson/Desktop/House Sales/Raw Data/King.geo.json'

In [7]:
# That's just in case you want to look at the JSON file contents here too:

f = open(r'/Users/kurtson/Desktop/House Sales/Raw Data/King.geo.json',)
  
# returns JSON object asa dictionary
data = json.load(f)
  
# Iterating through the json list
for i in data['features']:
    print(i)

{'type': 'Feature', 'properties': {'kind': 'county', 'name': 'King', 'state': 'WA'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[-121.2342, 47.7789], [-121.1192, 47.7789], [-121.0644, 47.7132], [-121.1301, 47.6748], [-121.1137, 47.5981], [-121.1137, 47.5981], [-121.2287, 47.5653], [-121.4642, 47.3681], [-121.4259, 47.286], [-121.3437, 47.2805], [-121.3656, 47.2257], [-121.3108, 47.2038], [-121.3054, 47.1381], [-121.4094, 47.1216], [-121.382, 47.0888], [-121.7873, 47.1764], [-121.9462, 47.1381], [-122.0941, 47.1928], [-122.1379, 47.2586], [-122.335, 47.2586], [-122.4172, 47.3188], [-122.3241, 47.3462], [-122.4227, 47.5762], [-122.3405, 47.5981], [-122.4281, 47.6584], [-122.3789, 47.7186], [-122.3953, 47.7789]]], [[[-122.5377, 47.3572], [-122.5377, 47.401], [-122.4829, 47.5105], [-122.4336, 47.4667], [-122.4391, 47.4064], [-122.3734, 47.39], [-122.4939, 47.3298]]]]}}


# 2. Data Wrangling

#### Fix dummy columns

In [8]:
# Select only the necessary columns and put them in a list called columns

columns =['latitude','longitude','grade','condition','bedrooms','bathrooms','price']


In [9]:
columns

['latitude',
 'longitude',
 'grade',
 'condition',
 'bedrooms',
 'bathrooms',
 'price']

In [10]:
# Create a subset

coordinates_df = df[columns]

In [11]:
coordinates_df.head()

,latitude,longitude,grade,condition,bedrooms,bathrooms,price
0,47.5112,-122.257,7,3,3,1.00,221900.0
1,47.7210,-122.319,7,3,3,2.25,538000.0
2,47.7379,-122.233,6,3,2,1.00,180000.0
3,47.5208,-122.393,7,5,4,3.00,604000.0
4,47.6168,-122.045,8,3,3,2.00,510000.0


In [12]:
df.head()

,Unnamed: 0,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,year_built,year_renovated,zipcode,latitude,longitude
0,0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045


In [13]:
#dropping Unnamed: 0 as it is unnecessary
df = df.drop(columns = ['Unnamed: 0'])

In [14]:
df.shape

(21613, 17)

### 2. Data wrangling 


In [15]:
# Create a data frame with just the values we want plotted

data_to_plot = coordinates_df[['condition','latitude', 'longitude']]
data_to_plot.head()

,condition,latitude,longitude
0,3,47.5112,-122.257
1,3,47.7210,-122.319
2,3,47.7379,-122.233
3,5,47.5208,-122.393
4,3,47.6168,-122.045


In [16]:
data_to_plot.condition.unique()

array([3, 5, 4, 1, 2])

In [17]:
data_to_plot.dtypes

condition      int64
latitude     float64
longitude    float64
dtype: object

In [19]:
# Setup a folium map at a high-level zoom
map = folium.Map(location = [37.0902, -95.7129], zoom_start = 5)

# Choropleth maps bind Pandas Data Frames and json geometries.This allows us to quickly visualize data combinations
folium.Choropleth(
    geo_data=country_geo,
    data=coordinates_df,
    columns=['longitude', 'latitude', 'condition'],
    key_on='feature.properties.kind',  # Replace with the correct path to your key
    fill_color='YlOrRd',
    fill_opacity=0.3,
    line_opacity=0.9,
    legend_name="condition").add_to(map)

# Add layer control
folium.LayerControl().add_to(map)

# Display the map
map

#### Unfortunately the Choropleth doesn't sync with the JSON to show the exact locations of the homes, however it does map out the region that we are focusing on. Folium can't provide this, however I can plug the coordinate information into tableau to assist with analysis further